In [1]:
from qpms import Particle, CTMatrix, lorentz_drude, EpsMuGenerator, TMatrixGenerator, BaseSpec, FinitePointGroup, ScatteringSystem, TMatrixInterpolator, EpsMu, dbgmsg_enable, dbgmsg_disable, dbgmsg_active, BesselType,eV, hbar, c
from qpms.symmetries import point_group_info
from qpms.cyewaldtest import scatsyswk_sswf_basis, scatsysw_eval_sswf
from qpms.cycommon import get_mn_y_scalar, get_nelem_scalar
import math
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
import numpy as np
eh = eV/hbar
np.random.seed(666)
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Rectangle, Circle
from mpl_toolkits.axes_grid1 import AxesGrid

def cleanarray(a, atol=1e-10, copy=True):
    a = np.array(a, copy=copy)
    sieve = abs(a.real) < atol
    a[sieve] = 1j * a[sieve].imag
    sieve = abs(a.imag) < atol
    a[sieve] = a[sieve].real
    return a

In [2]:
dbgmsg_enable(2)
#part_radius = 80e-9

<DebugFlags.THREADS|MISC: 3>

In [3]:
#modefile = '/u/46/necadam1/unix/repo/qpms/misc/modes_basis300nm_0nm__0nm_400nm_n1_b-2_k(0_0)um-1_cn150.npz' # PARAM!
modefile='/l/necadam1/test_mode_fieldplots/cyl_r50nm_h50nm_p560nmx560nm_mAu_n1.52_b-2_k(0_0)um-1_L3_cn200.npz'
#modefile = '/l/necadam1/sqlat-Tommi/sph_r80nm_p580nm_m5.0_n1.52_b-3_k(0_0)um-1_L3_cn3100.npz'
data = np.load(modefile, allow_pickle=True)
meta = data['meta'][()]
list(data.keys())

['meta',
 'empty_freqs',
 'ss_positions',
 'ss_fullvec_poffsets',
 'ss_fullvec_psizes',
 'ss_bspecs_flat',
 'ss_lattice_basis',
 'ss_reciprocal_basis',
 'eigval',
 'eigval_inside_metric',
 'eigvec',
 'residuals',
 'eigval_err',
 'ranktest_SV',
 'iri',
 'blochvector',
 'inside_contour',
 'refractive_index_internal']

In [4]:
# initialise the medium and dummy t-matrix generators, so we can "reconstruct" the ScatSystem instance (only background medium is relevant here)
medium = EpsMuGenerator(meta['background'])
emg_dummy = EpsMuGenerator(10)
tmg_dummy = TMatrixGenerator.sphere(medium, emg_dummy, 50e-9)
try: default_omega = data['eigval'][0]
except IndexError as ie:
    raise ValueError("Data file contains no eigenvalues") from ie
    

In [5]:
# create (periodic) ss, ssw; init wavevector to use with sswk constructor
particles = []
offsets = data['ss_fullvec_poffsets']
nelems = data['ss_fullvec_psizes']
for pi, pos in enumerate(data['ss_positions']):
    bspec = BaseSpec(data['ss_bspecs_flat'][offsets[pi]:offsets[pi]+nelems[pi]])
    particles.append(Particle(pos, tmg_dummy, bspec))
ss, ssw = ScatteringSystem.create(particles, medium, default_omega, latticebasis = data['ss_lattice_basis'])
wavevector = np.array([0., 0., 0.]) # it needs to be 3D for sswk "constructor", 
for i in range(len(meta['k'])):
    wavevector[i] = meta['k'][i]
    
# determine the "unit rectangle" size for plotting
lb = ss.lattice_basis
xmax = max(abs(lb[0,0]), abs(lb[1,0]))
ymax = max(abs(lb[0,1]), abs(lb[1,1]))
if (xmax > ymax):
    unitrect = (xmax, ss.unitcell_volume/xmax)
else:
    unitrect = (ss.unitcell_volume/ymax, ymax)

In [6]:
lMax_sswf = max(bspec.l()*2+1)
nelem_sswf = get_nelem_scalar(lMax_sswf)

In [7]:
# Create the finite version of ss, ssw
repeatrange = ((-20,21),(-20,21)) #PARAM
latshifts = [np.arange(*repeatrange[i])[:, None] * ss.lattice_basis[i] for i in (0,1)]
latpoints = latshifts[0][:,None,:] + latshifts[1][None,:,:]
latpoints.shape = (latpoints.shape[0]*latpoints.shape[1],3)

#latpoints = np.concatenate((latpoints[:220],latpoints[221:])) # DELETHE

particles_finite = []
for lp in latpoints:
    for pi, pos in enumerate(data['ss_positions']):
        bspec = BaseSpec(data['ss_bspecs_flat'][offsets[pi]:offsets[pi]+nelems[pi]])
        particles_finite.append(Particle(lp+pos, tmg_dummy, bspec))
ss_finite, ssw_finite = ScatteringSystem.create(particles_finite, medium, default_omega)


In [8]:
eigval_i = 0 # PARAM!
omega = data['eigval'][eigval_i]
fvc = data['eigvec'][eigval_i]
len_fvc = len(fvc)

In [9]:
ssw = ss(omega)
sswk = ssw._sswk(wavevector)

In [10]:
ssw_finite = ss_finite(omega)
fvc_finite = np.zeros((len_fvc * len(latpoints),), dtype=complex)

# THIS IS NOT GENERAL, but only for gamma point single-particle case!!! TODO FIXME!
for lpi, lp in enumerate(latpoints):
    phasefac = 1 # TODO
    fvc_finite[lpi*len_fvc:(lpi+1)*len_fvc] = fvc * phasefac


In [11]:
ss_finite.positions[220]

array([-8.4e-06, -2.8e-06,  0.0e+00])

In [12]:
ucfactor = 2.2 # "unit cell coverage"; unitrect sizes will be multiplied by this# PARAM!
zfactor = 5 # z range size / sqrt(unitcell_volume) size aspect ratio
resolution = 60 # PARAM!
sqrt_ucvol = math.sqrt(ss.unitcell_volume)
gridsz = [math.ceil(resolution * unitrect[i] / sqrt_ucvol) for i in (0,1)] #x, y
gridsz.append(math.ceil(zfactor / ucfactor * resolution)) # z
gridextent = [[ -(ucfactor * unitrect[i])/2, (ucfactor * unitrect[i] / 2) ] for i in (0,1)]
gridextent.append( [-(zfactor * sqrt_ucvol) / 2, (zfactor * sqrt_ucvol) / 2] )
gridlines = [np.linspace(*gridextent[i], gridsz[i]) for i in range(3)]
gridspacings = [gridlines[i][1]-gridlines[i][0] for i in range(3)] # should be very similar

gridoffsets = [0.,0.,0.] # PARAM!

xygrid = np.stack(np.meshgrid(gridlines[0], gridlines[1], gridoffsets[2], indexing='ij')).T
xygrid.shape=xygrid.shape[1:]
xygrid_s = xygrid - np.array([*gridspacings[:2], 0])/2 # correction of pcolormesh coordinates
xzgrid = np.stack(np.meshgrid(gridlines[0], gridoffsets[1], gridlines[2], indexing='ij')).T
xzgrid.shape = (gridsz[2], gridsz[0], 3)
xzgrid_s = xzgrid - np.array([gridspacings[0], 0, gridspacings[2]])/2

#for imshow
lenscale = 1000e-9 #PARAM!
xyextent = [gridextent[i][j]/lenscale for (i,j) in [(0,0),(0,1),(1,0),(1,1)]]
xzextent = [gridextent[i][j]/lenscale for (i,j) in [(0,0),(0,1),(2,0),(2,1)]]

In [13]:
ssw_finite.wavenumber

(11158313.604453105-3935.3064942051137j)

In [14]:
if False:
    logging.info("Start calculating xy field basis")
    xy_basis = np.zeros(xygrid.shape[:-1] + (len(bspec), 3), dtype=np.clongdouble)
    for pi in range(ss_finite.particle_count):
        xy_basis += ssw_finite.scattered_field_basis(xygrid, particle=pi)
    logging.info("Start calculating xz field basis")
    xz_basis = np.zeros(xzgrid.shape[:-1] + (len(bspec), 3), dtype=np.clongdouble)
    for pi in range(ss_finite.particle_count):
        xz_basis += ssw_finite.scattered_field_basis(xzgrid, particle=pi)
else:
    logging.info("Start calculating xy field basis")
    xy_basis = np.zeros(xygrid.shape[:-1] + (nelem_sswf,), dtype=np.clongdouble)
    kcomp = np.zeros(xygrid.shape[:-1] + (nelem_sswf,), dtype=np.clongdouble)
    cinp = np.zeros(xygrid.shape[:-1] + (nelem_sswf,), dtype=np.clongdouble)
    nsum = np.zeros(xygrid.shape[:-1] + (nelem_sswf,), dtype=np.clongdouble)
    for pi in range(ss_finite.particle_count):
        cinp[...] = scatsysw_eval_sswf(ssw_finite, xygrid-ss_finite.positions[pi])
        #cinp[...] = ssw_finite.scattered_field_basis(xygrid, particle=pi) - kcomp
        nsum[...] = xy_basis + cinp
        kcomp[...] = (nsum - xy_basis) - cinp
        xy_basis[...] = nsum
    #logging.info("Start calculating xz field basis")
    #xz_basis = np.zeros(xzgrid.shape[:-1] + (len(bspec), 3), dtype=np.clongdouble)
    #kcomp = np.zeros(xzgrid.shape[:-1] + (len(bspec), 3), dtype=np.clongdouble)
    #cinp = np.zeros(xzgrid.shape[:-1] + (len(bspec), 3), dtype=np.clongdouble)
    #nsum = np.zeros(xzgrid.shape[:-1] + (len(bspec), 3), dtype=np.clongdouble)
    #for pi in range(ss_finite.particle_count):
    #    cinp[...] = ssw_finite.scattered_field_basis(xzgrid, particle=pi) - kcomp
    #    nsum[...] = xz_basis + cinp
    #    kcomp[...] = (nsum - xz_basis) - cinp
    #    xz_basis[...] = nsum
logging.info("Basis fields done")


2020-11-24 00:10:13,253 Start calculating xy field basis
2020-11-24 00:10:46,003 Basis fields done


In [19]:

xy_sswf=np.array(xy_basis, dtype=np.cdouble)
#xz_basis=np.array(xz_basis, dtype=np.cdouble)

In [16]:
cutoff_ratio_default = 0.03
def cutfield(a, cutoff_ratio=cutoff_ratio_default):
    vmin = np.nanquantile(a, cutoff_ratio)
    vmax = np.nanquantile(a, 1-cutoff_ratio)
    return np.clip(a, vmin, vmax)
    

In [20]:
%matplotlib inline
plotfile = "/tmp/basisfields_sswf_%dx%d_kahanlong.pdf" % (repeatrange[0][1]-repeatrange[0][0], repeatrange[1][1]-repeatrange[1][0]) # PARAM

lMax_sswf = max(bspec.l()*2+1)
nelem_sswf = get_nelem_scalar(lMax_sswf)
m, l = get_mn_y_scalar(lMax_sswf)
with PdfPages(plotfile) as pdf:
  for y in range(nelem_sswf):
    xyf = cleanarray(xy_sswf[...,y])
    
    f = plt.figure(figsize=(7,4))
    f.suptitle("(%d, %d)" % (l[y], m[y]))
    grid = AxesGrid(f, 111,  # similar to subplot(111)
                nrows_ncols=(1, 2),
                axes_pad=(0.45, 0.65),
                label_mode="L",
                share_all=False,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="7%",
                cbar_pad="2%",
                )
    
    # xy part
    ax = grid[0:2]
    cbax = grid.cbar_axes[0:2]

    for axi in range(2):
        ax[axi].set_xlim(xyextent[:2])
        ax[axi].set_ylim(xyextent[2:])
    flim = max(np.amax(cutfield(abs(xyf[...].real))), np.amax(cutfield(abs(xyf[...].imag)))) #this is stupid
    print(flim)
    if (flim != 0):
        axi=0
        c = ax[axi].imshow(cutfield(xyf[...].real), origin='lower', cmap=cm.bwr, extent=xyextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)
        axi=1
        c = ax[axi].imshow(cutfield(xyf[...].imag), origin='lower', cmap=cm.bwr, extent=xyextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)
    
    pdf.savefig(f)
    plt.close(f)

6.0524498526083335
4.2434506677760275
0.0
4.2434506677760275
74.49131219111493
0.0
17.125287530925565
0.0
74.49131219111496
3033.596208170187
0.0
291.3228792370602
0.0
291.3228792370604
0.0
3033.596208170185
325442.9421631311
0.0
36367.30388638537
0.0
2593.363573767691
0.0
36367.30388638537
0.0
325442.94216313114
55258511.734574094
3.2446822675197457e-09
4100943.9691264187
3.0046693518433233e-10
116965.00569788122
0.0
116965.0056978812
3.0046693518433233e-10
4100943.9691264187
3.2446822675197457e-09
55258511.734574094
33211060807.16342
8.706164186008641e-07
584177664.3193926
5.3957292810118145e-08
31098454.58973242
1.2683631739645677e-09
1130182.3913984648
1.2683631739645685e-09
31098454.589732435
5.3957292810118145e-08
584177664.3193926
8.706164186008643e-07
33211060807.16345
7360815146862.141
0.0008115114770172084
186387080793.09515
1.2201637620488454e-05
6463313293.489
5.575428582508802e-07
102359500.42227368
1.449354852521083e-08
102359500.4222737
5.575428582508805e-07
6463313293.4